# Working with Jaxfluids

(Introductory) notebook on working with [Jaxfluids](https://github.com/adopt-opt/jaxfluids) in the context of design optimization. For installation instructions of the underlying JAX package, please see their [GitHub repository](https://github.com/google/jax).

## Manipulation of JSON

For the manipulation of the JSON files we want to use [PyJSON](https://github.com/niyoh120/pyjson), with which we can then alter the entries of our JSON files.

In [4]:
import pyjson
import json

Storing the path to our two most important JSON files into variables

In [27]:
case_setup = './cylinderflow.json'
num_setup = './numerical_setup.json'

Altering the JSON file

In [26]:
jsonFile = open(case_setup, "r") # Open the JSON file for reading
setup = json.load(jsonFile) # Read the JSON into the buffer
jsonFile.close() # Close the JSON file

## Working with buffered content
setup["domain"]["x"]["cells"] = 75
setup["domain"]["x"]["range"] = [-2, 4]
setup["domain"]["y"]["cells"] = 50
setup["domain"]["y"]["range"] = [-2, 2]

## Save our changes to JSON file
jsonFile = open(case_setup, "w+")
jsonFile.write(json.dumps(setup))
jsonFile.close()

The same approach can be replicated for other tasks such as changing the lambda function determining the shape of the body inside of the domain for design optimization purposes

```json
{
"initial_condition": {
        "primes":{
            "rho": 1.0,
            "u": 0.0,
            "v": 0.0,
            "w": 0.0,
            "p": 1.0
        },
        "levelset": "lambda x,y: - 0.1 + jnp.sqrt(x**2 + y**2)"
    }
}
```

## Ahead-of-Time (AOT) Compilation with JAX

## Batch Evaluation with AOT'd Kernels